In [5]:
#Installing packages onto the device
#!{sys.executable} -m pip install -U selenium

import sys

#Define the folder with the python scripts for web scraping in order to import these scripts
sys.path.insert(0, 'C:\\Users\\AU451FE\\OneDrive - EY\\Desktop\\Python\\HSreplay_scraper\\Scripts')
from Analyzer import DeckAnalyzer as DA
from Selector import DeckSelector as DS

#External browser Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


#Other useful packages
import sys
from bs4 import BeautifulSoup
import requests
import time
from datetime import date
import datetime
import pandas as pd
import numpy as np
import re #String search
import warnings

#Silence the deprecation warning when minimizing the external drivers
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [6]:
driver_path = 'C:/Users/AU451FE/OneDrive - EY/Desktop/Python/HSreplay_scraper/chromedriver'
#deck_code = 'TDJ576DqVJdHynQToOkWtb'
class_name = 'Warrior'
class_archetype = '222'

In [8]:
class DeckSelector:
    '''Define the class and archetype of a deck at hsreplay.net and return the analysis of the decks of said archetype
        featured on that website
        
        deck_class = name of the class in quotes (e.g. 'Druid')
        deck_archetype = code for the archetype in quotes (e.g. '372')
    '''
    def __init__(self, deck_class, deck_archetype):
        deck_class = deck_class.upper()    #Converting the deck_class into upper case
        self.url = f'https://hsreplay.net/decks/#playerClasses={deck_class}&gameType=RANKED_STANDARD&archetypes={deck_archetype}'
  
        options = webdriver.ChromeOptions()
        options.set_headless(True) 
        
        
        self.driver = webdriver.Chrome(driver_path,options=options)
        self.driver.get(self.url) 
        
        self.driver.maximize_window()      #Maximize the window

        #Closing the privacy settings window
        try:
            WebDriverWait(self.driver, 10).until(lambda x: x.find_element_by_class_name('css-flk0bs'))
            self.driver.find_element_by_class_name('css-flk0bs').click()
        except TimeoutException:
            raise Exception('The privacy window has not shown up; try running the script again')
        
        
        #Looks for the codes of the specified archetype
        list_decks = self.driver.find_elements_by_class_name('deck-tile')
        
        codes = []
        for l in list_decks:
            link = l.get_attribute('href')

            try:
                m = re.search('decks/(.+?)/#game', link).group(1)

            except AttributeError:
                print(f'The entered URL is invalid')
                break
            codes.append(m)

        self.codes = codes
        print('The codes are stored under the attribute self.codes')
        
        #Get the archetype name
        self.archetype_name = self.driver.find_element_by_xpath('//*[@id="decks-container"]/main/div[3]/section/ul/li[2]/a/div/div[1]/h3').text
        self.driver.quit()
    
    def get_overviews_df(self):
        '''Return a data frame with an overview of all decks of a given archetype
        '''
        overviews_df = pd.DataFrame()
        for c in self.codes:
            D = DA(driver_path, c)
            df = D.get_overview_df()
            overviews_df = pd.concat([overviews_df, df], axis = 0)
            
            deck_position = self.codes.index(c) + 1
            print(f'Generated data for {deck_position}/{len(self.codes)} decks of archetype {self.archetype_name}')
            
        overviews_df = overviews_df.reset_index()
        return overviews_df
    
    def write_to_excel(self, today = date.today().strftime("%m-%d")):
        df = self.get_overviews_df()
        
        with pd.ExcelWriter(f'C:/Users/AU451FE/OneDrive - EY/Desktop/Python/HSreplay_Scraper/Data Frames/{self.archetype_name} {today}.xlsx') as writer:
            df.to_excel(writer, sheet_name = 'Overview')



In [10]:
S = DeckSelector(class_name, class_archetype)

The codes are stored under the attribute self.codes


In [14]:
#codes_here = ['sUGg1miAk6Drk72nX1uhhc',
# 'L5ODjRqeQI5mFk8P6i1mhe']

In [15]:
start = time.time()
output = S.write_to_excel()
end = time.time()
print('Time taken in seconds:', end - start)

Generating the overview
Generated data for 1/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 2/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 3/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 4/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 5/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 6/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 7/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 8/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 9/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 10/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 11/18 decks of archetype Rush Warrior
Generating the overview
Generated data for 12/18 decks of archetype Rush Warrior
Generating the overview
Generated dat